# Central Pattern Generator for control of quadruped robot

This is part 1 of the Legged robot course's project \#2. We will focus on implementing a montion controller based on a Central Pattern Generato (CPG).

## Importing the required libraries

In [1]:
import time
import numpy as np # numpy library for matrix computatios

from env.quadruped_gym_env import QuadrupedGymEnv
from hopf_network import HopfNetwork

%load_ext autoreload
%autoreload 2

## Note on this notebook vs the hopf_network module

Instead of directly implementing the simulation in the hopf_network module's default function __\_\_main\_\___, we will work it out here in this current notebook. 

This will enable us to test different parameters, perform different graphs, in a reproducible and illustrated step by step fashion. The steps will be explained as needed. 

Finally, we will move the chosen parameters and code to the hopf_network module's default function __\_\_main\_\___, such that it is a standalone module. 

In [2]:
# Initialise dynamic engine of pybullet and we connect it to the Pybullet GUI
ADD_CARTESIAN_PD = True
TIME_STEP = 0.001
foot_y = 0.0838 # this is the hip length 
sideSign = np.array([-1, 1, -1, 1]) # get correct hip sign (body right is negative)

env = QuadrupedGymEnv(
                  render=True,              # visualize
                  on_rack=False,              # useful for debugging! 
                  isRLGymInterface=False,     # not using RL
                  time_step=TIME_STEP,
                  action_repeat=1,
                  motor_control_mode="TORQUE",
                  add_noise=False,    # start in ideal conditions
                  # record_video=True
                  )

/usr/local/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [3]:
# After implementing the gait offsets, let's check that everything looks ok (divide by 2pi for clarity)
# initialize Hopf Network, supply gait
print(HopfNetwork(gait="TROT").PHI/(2*np.pi))
print(HopfNetwork(gait="PACE").PHI/(2*np.pi))
print(HopfNetwork(gait="BOUND").PHI/(2*np.pi))
print(HopfNetwork(gait="WALK").PHI/(2*np.pi))

TROT
[[ 0.  -0.5 -0.5  0. ]
 [ 0.5  0.   0.   0.5]
 [ 0.5  0.   0.   0.5]
 [ 0.  -0.5 -0.5  0. ]]
PACE
[[ 0.  -0.5  0.  -0.5]
 [ 0.5  0.   0.5  0. ]
 [ 0.  -0.5  0.  -0.5]
 [ 0.5  0.   0.5  0. ]]
BOUND
[[ 0.   0.  -0.5 -0.5]
 [ 0.   0.  -0.5 -0.5]
 [ 0.5  0.5  0.   0. ]
 [ 0.5  0.5  0.   0. ]]
WALK
[[ 0.   -0.5  -0.25  0.25]
 [ 0.5   0.    0.25  0.75]
 [ 0.25 -0.25  0.    0.5 ]
 [-0.25 -0.75 -0.5   0.  ]]


Which seems right !

In [11]:
# Testing the simulation works, the robot does move according to gravity and small torques...
env.reset()
for i in range(1000):
    env.step(action = np.zeros(12) + 5 ) 

Everything seems to work fine, although there were troubles at the beginning on macOS because of pyqt5.

# TO BE CONTINUED

In [4]:
# initialize Hopf Network, supply gait
cpg = HopfNetwork(
                mu=1**2,                # converge to sqrt(mu)
                omega_swing=1*2*np.pi,  # MUST EDIT
                omega_stance=1*2*np.pi, # MUST EDIT
                gait="TROT",            # change depending on desired gait
                coupling_strength=1,    # coefficient to multiply coupling matrix
                couple=True,            # should couple
                time_step=TIME_STEP,        # time step 
                ground_clearance=0.05,  # foot swing height 
                ground_penetration=0.01,# foot stance penetration into ground 
                robot_height=0.25,      # in nominal case (standing) 
                des_step_len=0.04      # desired step length 
                )

TEST_STEPS = int(10 / (TIME_STEP))
t = np.arange(TEST_STEPS)*TIME_STEP

# [TODO] initialize data structures to save CPG and robot states



TROT


In [ ]:
############## Sample Gains
# joint PD gains
kp=np.array([150,70,70])
kd=np.array([2,0.5,0.5])
# Cartesian PD gains
kpCartesian = np.diag([2500]*3)
kdCartesian = np.diag([40]*3)


for j in range(TEST_STEPS):
    # initialize torque array to send to motors
    action = np.zeros(12) 
    # get desired foot positions from CPG 
    xs,zs = cpg.update()
    # [TODO] get current motor angles and velocities for joint PD, see GetMotorAngles(), GetMotorVelocities() in quadruped.py
    # q = 
    # dq = 

    # loop through desired foot positions and calculate torques
    for i in range(4):
        # initialize torques for legi
        tau = np.zeros(3)
        # get desired foot i pos (xi, yi, zi) in leg frame
        leg_xyz = np.array([xs[i],sideSign[i] * foot_y,zs[i]])
        # call inverse kinematics to get corresponding joint angles (see ComputeInverseKinematics() in quadruped.py)
        leg_q = np.zeros(3) # [TODO] 
        # Add joint PD contribution to tau for leg i (Equation 4)
        tau += np.zeros(3) # [TODO] 

        # add Cartesian PD contribution
        if ADD_CARTESIAN_PD:
            # Get current Jacobian and foot position in leg frame (see ComputeJacobianAndPosition() in quadruped.py)
            # [TODO] 
            # Get current foot velocity in leg frame (Equation 2)
            # [TODO] 
            # Calculate torque contribution from Cartesian PD (Equation 5) [Make sure you are using matrix multiplications]
            tau += np.zeros(3) # [TODO]

        # Set tau for legi in action vector
        action[3*i:3*i+3] = tau

    # send torques to robot and simulate TIME_STEP seconds 
    env.step(action) 

    # [TODO] save any CPG or robot states


In [ ]:
##################################################### 
# PLOTS
#####################################################
# example
# fig = plt.figure()
# plt.plot(t,joint_pos[1,:], label='FR thigh')
# plt.legend()
# plt.show()

In [ ]:
# time.sleep(2)
pybullet.disconnect()